In [1]:
! pip install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 949 kB/s eta 0:00:01


# Module

In [1]:
#Import Library
import random
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow with Keras.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()

# Dataset

In [2]:
# MNIST 데이터셋 준비

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()   

# 각 데이터 크기 출력
print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"x_test.shape: {x_test.shape}")
print(f"y_test.shape: {y_test.shape}")

x_train.shape: (60000, 28, 28)
y_train.shape: (60000,)
x_test.shape: (10000, 28, 28)
y_test.shape: (10000,)


# Preprocessing

In [3]:
# Normalization - x
print('최소값:',np.min(x_train), ' 최대값:',np.max(x_train))

x_train_norm = x_train.astype('float32')/255.0
x_test_norm = x_test.astype('float32') / 255.0

print('최소값:',np.min(x_train_norm), ' 최대값:',np.max(x_train_norm))
#print(f"x_train.shape: {x_train.shape}")

최소값: 0  최대값: 255
최소값: 0.0  최대값: 1.0


In [4]:
# Normalization - y (One-hot-encoding)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Model

In [5]:
from keras.layers import Input,Conv2D,Dropout, Dense,Flatten,GlobalAveragePooling2D,MaxPool2D
from keras.models import Model

#  tf.keras의 Sequential API를 이용하여 LeNet 모델 생성
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25632     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [7]:
# 채널수 맞추기
print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))

x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)

print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))


Before Reshape - x_train_norm shape: (60000, 28, 28)
Before Reshape - x_test_norm shape: (10000, 28, 28)
After Reshape - x_train_reshaped shape: (60000, 28, 28, 1)
After Reshape - x_test_reshaped shape: (10000, 28, 28, 1)


# Train Model

In [8]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])


In [9]:
%%time
model.fit(x_train_reshaped, y_train, epochs=10)


Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 126us/sample - loss: 0.1911 - accuracy: 0.9417
Epoch 2/10
60000/60000 [==============================] - 4s 71us/sample - loss: 0.0616 - accuracy: 0.9813
Epoch 3/10
60000/60000 [==============================] - 4s 65us/sample - loss: 0.0458 - accuracy: 0.9859
Epoch 4/10
60000/60000 [==============================] - 5s 76us/sample - loss: 0.0370 - accuracy: 0.9884
Epoch 5/10
60000/60000 [==============================] - 6s 104us/sample - loss: 0.0295 - accuracy: 0.9905
Epoch 6/10
60000/60000 [==============================] - 5s 85us/sample - loss: 0.0240 - accuracy: 0.9924
Epoch 7/10
60000/60000 [==============================] - 5s 85us/sample - loss: 0.0208 - accuracy: 0.9933
Epoch 8/10
60000/60000 [==============================] - 5s 87us/sample - loss: 0.0173 - accuracy: 0.9943
Epoch 9/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0144 - accuracy: 0.9952
Epoch 10/10


# Model Evaluation

In [10]:
predictions = model.predict(x_test_reshaped)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test example: {}%'.format(accuracy*100))

/home/aiffel/anaconda3/envs/aiffel2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Accuracy on benign test example: 99.0%


# Adversarial Examples

In [11]:
# ART
import art
from art.estimators.classification import KerasClassifier

#Wrap model with ART Keras Classifier
estimator = KerasClassifier(model=model, clip_values=(0,1), use_logits=True)

# 1. Boundary Attack

### Prediction_1

In [21]:
%%time
from art.attacks.evasion import BoundaryAttack

attack_BA = BoundaryAttack(estimator,batch_size=200)

#Create Adversarial Examples
examples_BA = attack_BA.generate(x_test_reshaped,y_test)

Boundary attack:   0%|          | 0/10000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

...

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

CPU times: user 12min 16s, sys: 22.4 s, total: 12min 38s
Wall time: 10min 41s


In [22]:
prediction = model.predict(examples_BA)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 99.52%


In [23]:
%%time
from art.attacks.evasion import BoundaryAttack

attack_BA = BoundaryAttack(estimator,batch_size=50)

#Create Adversarial Examples
examples_BA = attack_BA.generate(x_test_reshaped,y_test)

Boundary attack:   0%|          | 0/10000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

...

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/5000 [00:00<?, ?it/s]

CPU times: user 11min 54s, sys: 18.1 s, total: 12min 12s
Wall time: 10min 21s


In [24]:
prediction = model.predict(examples_BA)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 99.52%


# 2. DeepFool

In [16]:
%%time
from art.attacks.evasion import DeepFool

attack_DF = DeepFool(estimator,batch_size=200)

#Create Adversarial Examples
examples_DF = attack_DF.generate(x_test_reshaped,y_test)

DeepFool:   0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 1min 46s, sys: 4.12 s, total: 1min 50s
Wall time: 1min 33s


In [17]:
prediction = model.predict(examples_DF)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 5.74%


# 3. Basic Iterative Method ( BIM )

In [20]:
%%time
from art.attacks.evasion import BasicIterativeMethod

attack_BIM = BasicIterativeMethod(estimator,eps=0.1,batch_size=200)

#Create Adversarial Examples
examples_BIM = attack_BIM.generate(x_test_reshaped,y_test)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 36.7 s, sys: 970 ms, total: 37.6 s
Wall time: 33.7 s


In [21]:
prediction = model.predict(examples_BIM)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 72.19%


# 4. Zoo (Black-box)-targeted

In [25]:
import random
y_test2 = y_test.copy()
np.random.shuffle(y_test2)

In [26]:
y_test2_norm = tf.keras.utils.to_categorical(y_test2, 10)

In [27]:
%%time
from art.attacks.evasion import ZooAttack

attack_zoo = ZooAttack(estimator,batch_size=200)

#Create Adversarial Examples
x_attack_zoo = attack_zoo.generate(x_test_reshaped)

ZOO:   0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 3h 32min 37s, sys: 33.5 s, total: 3h 33min 10s
Wall time: 3h 32min 51s


In [28]:
loss,acc = model.evaluate(x_attack_zoo,y_test2)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

/home/aiffel/anaconda3/envs/aiffel2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Accuracy on Adversarial Exmaples: 10.39000004529953%


# 5. Square Attack

In [25]:
%%time
from art.attacks.evasion import SquareAttack

attack_SA = SquareAttack(estimator,eps=0.1,batch_size=200)

#Create Adversarial Examples
examples_SA = attack_SA.generate(x_test_reshaped,y_test)

SquareAttack - restarts:   0%|          | 0/1 [00:00<?, ?it/s]

SquareAttack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 34.4 s, sys: 41.9 s, total: 1min 16s
Wall time: 1min 55s


In [26]:
prediction = model.predict(examples_SA)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 95.73%


In [27]:
%%time
from art.attacks.evasion import SquareAttack

attack_SA = SquareAttack(estimator,eps=0.3,batch_size=200)

#Create Adversarial Examples
examples_SA = attack_SA.generate(x_test_reshaped,y_test)

SquareAttack - restarts:   0%|          | 0/1 [00:00<?, ?it/s]

SquareAttack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 21.1 s, sys: 30.1 s, total: 51.2 s
Wall time: 1min 21s


In [28]:
prediction = model.predict(examples_SA)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 20.13%


# 6. Hop Skip Jump

In [15]:
%%time
from art.attacks.evasion import HopSkipJump

attack_hop = HopSkipJump(estimator,batch_size=200,max_iter=10)

#Create Adversarial Examples
examples_hop = attack_hop.generate(x_test_reshaped,y_test)

HopSkipJump:   0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 46min 12s, sys: 1min 34s, total: 47min 46s
Wall time: 38min 35s


In [26]:
prediction = model.predict(examples_hop)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 95.73%


# 7. Elastic Net

In [20]:
%%time
from art.attacks.evasion import ElasticNet

attack_hop = ElasticNet(estimator,batch_size=200)

#Create Adversarial Examples
examples_hop = attack_hop.generate(x_test_reshaped,y_test)

EAD:   0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 44min 44s, sys: 1min 44s, total: 46min 29s
Wall time: 39min 6s


In [26]:
prediction = model.predict(examples_hop)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 95.73%


# Zoo

In [20]:
%%time
from art.attacks.evasion import ZooAttack

attack_hop = ElasticNet(estimator,batch_size=200)

#Create Adversarial Examples
examples_hop = attack_hop.generate(x_test_reshaped,y_test)

EAD:   0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 44min 44s, sys: 1min 44s, total: 46min 29s
Wall time: 39min 6s


In [26]:
prediction = model.predict(examples_hop)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 95.73%
